In [5]:
# slow down a bit when hacking something together, e.g. I forgot to add a simple function call 
# tuple unpacking is nice, but cannot be done in a nested list comprehension
# don't forget .items in  for k,v in dict.items()
# use hashlib for md5 encodings
# multiline list comprehensions don't need extra parentheses, but multiline if statements do
# np.clip min and max can be omitted by specifying None
# try except looks nice untill it obscures your real error
# parsing ints to ints instead of strings is really important
# checking whether someting is an int should be done with isinstance, not with isalpha() (fails on int)
# removing from a list while iterating can be done safely by iterating over a slice(?)
# with re make sure to use r'' literal strings
# read assignment before tinkering with networkx and discovering its not necessary
# sometimes a simple for loop works better then a list comprehension when parsing the input, and just add to concept variables
# for incrementing a string, you can use chr(ord(inp)+1)
# find repeating characters re.findall(r'([a-z])\1', password)
# regex: modify operator to nongreedy by appending ?
# ok so sometime you can bruteforce a problem....
# while manually modifying the input, make sure to change exactly the right line

In [12]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
import itertools
import numpy as np
from matplotlib import pyplot as plt
from aocutils.common import data
import networkx as nx
from itertools import permutations
import re


In [13]:
# solution with numpy array
(medicine, start) = data('input', parser=str, sep='\n\n')
conv = set()
for m in medicine.split('\n'):
    (inp, out) = m.split(' => ')
    conv.add((inp,out))

In [22]:
outcomes = set()
for inp, out in conv:
    poss = re.finditer(inp, start)
    for p in poss:
        i = p.start()
        outcomes.add(start[0:i] + out + start[i+ len(inp):])
len(outcomes)

535

In [5]:
conv = {end: begin for (begin, end) in conv}

In [6]:
# bfs approach failed miserably, way too many options
unique = ['Si', 'Th', 'Ar', 'CRn', 'Y']
target = 'PTiTiTiTiBCaCaSiRnCaCaFYFAr'
# target = 'HOHOHO'
targetcount = {u:target.count(u) for u in unique}

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start 
        start += len(sub)
        
def get_neigh(start):
    outcomes = set()
    for c in conv:
        inp, out = c
        for i in find_all(start, inp):
            # print(i,inp)
            new = start[0:i] + out + start[i+ len(inp):]

            if len(new)<=len(target) and all(new.count(k) <= v for k,v in targetcount.items()) and new.endswith('Ar') and new.count('Y')==targetcount['Y']:
                outcomes.add(new)
    return outcomes
ar_options = {k for k,v in conv.items() if v.endswith('Ar')}
for start in ['Ca']:
    print('running',start)
    print(start, bfs(get_neigh, start, goal=target) )

running Ca


NameError: name 'bfs' is not defined

In [7]:
def findmatch(target):
    for i in range(10,0,-1):
        if target[:i] in conv:
            # print('found', target[:i])
            return target[:i]
    else:
        if len(target)==1:
            return False
        return findmatch(target[1:])

def replace(to_replace, target):
    return target.replace(to_replace, conv[to_replace], 1 )

def convert_substring(target):
    # returns count and new molecule

    for i in range(0,10000):
        if target in conv.values():
            print('done',target,i)

            return target, i
        to_replace = findmatch(target)
        if not to_replace:
            return False
        target = replace(to_replace, target)

target = 'SiRnFAr'
convert_substring(target)


done P 1


('P', 1)

In [8]:
target = 'CRnCaCaCaSiRnBPTiMgArSiRnSiRnMgArSiRnCaFArTiTiBSiThFYCaFArCaCaSiThCaPBSiThSiThCaCaPTiRnPBSiThRnFArArCaCaSiThCaSiThSiRnMgArCaPTiBPRnFArSiThCaSiRnFArBCaSiRnCaPRnFArPMgYCaFArCaPTiTiTiBPBSiThCaPTiBPBSiRnFArBPBSiRnCaFArBPRnSiRnFArRnSiRnBFArCaFArCaCaCaSiThSiThCaCaPBPTiTiRnFArCaPTiBSiAlArPBCaCaCaCaCaSiRnMgArCaSiThFArThCaSiThCaSiRnCaFYCaSiRnFYFArFArCaSiRnFYFArCaSiRnBPMgArSiThPRnFArCaSiRnFArTiRnSiRnFYFArCaSiRnBFArCaSiRnTiMgArSiThCaSiThCaFArPRnFArSiRnFArTiTiTiTiBCaCaSiRnCaCaFYFArSiThCaPTiBPTiBCaSiThSiRnMgArCaF'


In [9]:
count = 0
for j in range(100):
    endindex = target.find('Ar')+2
    beginindex = 0
    found = 0
    for i in range(endindex-1,0,-1):
        if target[i:i+2]=='Rn':
            print('found Rn at',i)
            found +=1
        if found == 2:
            beginindex = i+2
            break
    print('indices to start', beginindex, endindex)
    worked = False
    while not worked:
        print('indices', beginindex, endindex)
        print('to replace', target[beginindex:endindex])
        res = convert_substring(target[beginindex:endindex])
        if res: 
            worked = True
        else:
            beginindex+=1


    replacement, addition = res
    count += addition
    target = target[0:beginindex] + replacement + target[endindex:]
    print('len', len(target), 'count', count)
    print(target)
    assert target.count('Rn')== target.count('Ar')

found Rn at 11
found Rn at 1
indices to start 3 21
indices 3 21
to replace CaCaCaSiRnBPTiMgAr
done Ca 7
len 489 count 7
CRnCaSiRnSiRnMgArSiRnCaFArTiTiBSiThFYCaFArCaCaSiThCaPBSiThSiThCaCaPTiRnPBSiThRnFArArCaCaSiThCaSiThSiRnMgArCaPTiBPRnFArSiThCaSiRnFArBCaSiRnCaPRnFArPMgYCaFArCaPTiTiTiBPBSiThCaPTiBPBSiRnFArBPBSiRnCaFArBPRnSiRnFArRnSiRnBFArCaFArCaCaCaSiThSiThCaCaPBPTiTiRnFArCaPTiBSiAlArPBCaCaCaCaCaSiRnMgArCaSiThFArThCaSiThCaSiRnCaFYCaSiRnFYFArFArCaSiRnFYFArCaSiRnBPMgArSiThPRnFArCaSiRnFArTiRnSiRnFYFArCaSiRnBFArCaSiRnTiMgArSiThCaSiThCaFArPRnFArSiRnFArTiTiTiTiBCaCaSiRnCaCaFYFArSiThCaPTiBPTiBCaSiThSiRnMgArCaF
found Rn at 11
found Rn at 7
indices to start 9 17
indices 9 17
to replace SiRnMgAr
done Ca 1
len 483 count 8
CRnCaSiRnCaSiRnCaFArTiTiBSiThFYCaFArCaCaSiThCaPBSiThSiThCaCaPTiRnPBSiThRnFArArCaCaSiThCaSiThSiRnMgArCaPTiBPRnFArSiThCaSiRnFArBCaSiRnCaPRnFArPMgYCaFArCaPTiTiTiBPBSiThCaPTiBPBSiRnFArBPBSiRnCaFArBPRnSiRnFArRnSiRnBFArCaFArCaCaCaSiThSiThCaCaPBPTiTiRnFArCaPTiBSiAlArPBCaCaCaCaCaSiRnMgAr

In [10]:
# and from HCaf 2 more mutations to e = 210+2